# Export Simulation Results

In [1]:
## Load the BPTK Package
from BPTK_Py.bptk import bptk 

bptk = bptk()

In [2]:
bptk.register_scenarios(scenario_manager="smCustomerAcquisition",scenarios={
                          "interactiveScenario":{
                              "constants":{
                                 "referrals":0,
                                  "advertisingSuccessPct":0.1,
                                  "referralFreeMonths":3,
                                  "referralProgamAdoptionPct":10
                                }
                          }
}
                    
)

In [3]:
bptk.list_scenarios(["smCustomerAcquisition"])


*** smCustomerAcquisition ***
	 base
	 serviceFlop
	 rethinkAdvertising
	 referSomeonePlease
	 hereWeGo
	 boomButBust
	 interactiveScenario


In [5]:
%%time
bptk.export_scenarios(
    scenario_manager="smCustomerAcquisition",
    equations=["customers","profit"],
    filename='/Users/olivergrasl/Desktop/test.xlsx',
    interactive_scenario="interactiveScenario",
    interactive_equations=["customers","profit"],
    interactive_settings= {
        "advertisingSuccessPct":(0,0.2,0.01),
        "referralFreeMonths":(0,40,10),
        "referralProgramAdoptionPct":(0,12,1),
        "referrals":(0,12,1)
    }
)

CPU times: user 3min 47s, sys: 7.51 s, total: 3min 55s
Wall time: 4min 11s


In [46]:
from IPython.display import IFrame       
IFrame('https://app.powerbi.com/view?r=eyJrIjoiNDFlMTA2ZGItYmFkOC00Mjg3LTljODYtYzRkYzY2YmFmM2E4IiwidCI6ImZjNWRmZDc1LTQ1NzktNGQ3MC05YTE3LTk0MmQ0OWMwMTc2ZiIsImMiOjl9', width=933, height=700)